<a href="https://colab.research.google.com/github/NganVoThi/Face-Recognition-using-GenerativeAI-and-TensorFlowLite/blob/main/MobileNetV2_Trainning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mtcnn
!pip install tensorflow
!pip install matplotlib
!pip install pillow scikit-learn
!pip install tensorflow_model_optimization

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 37.9 MB/s eta 0:00:00


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


1. Trích xuất ảnh từ khuôn mặt

In [ ]:
import numpy as np
import os
import cv2
from mtcnn import MTCNN

# Đường dẫn đến thư mục chứa ảnh
input_folder = '/content/drive/MyDrive/DATN/images'
output_folder = '/content/drive/MyDrive/DATN/images_face'

# Tạo detector MTCNN
detector = MTCNN()

# Hàm phát hiện và cắt khuôn mặt
def detect_and_crop_face(img):
    # Chuyển ảnh sang RGB vì MTCNN yêu cầu ảnh RGB
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Phát hiện khuôn mặt
    faces = detector.detect_faces(img_rgb)

    # Nếu phát hiện khuôn mặt, cắt và trả về
    if len(faces) > 0:
        # Lấy thông tin box của khuôn mặt đầu tiên (có thể thay đổi để xử lý nhiều khuôn mặt)
        x, y, w, h = faces[0]['box']
        x, y = max(0, x), max(0, y)  # Đảm bảo tọa độ không âm
        face_img = img[y:y+h, x:x+w]  # Cắt vùng mặt
        return face_img
    return None  # Nếu không phát hiện mặt, trả về None

# Hàm xử lý toàn bộ ảnh trong thư mục gốc và các thư mục con
def process_images(input_dir, output_dir):
    for root, dirs, files in os.walk(input_dir):
        # Tạo đường dẫn output tương ứng với mỗi thư mục con
        relative_path = os.path.relpath(root, input_dir)
        target_folder = os.path.join(output_dir, relative_path)
        os.makedirs(target_folder, exist_ok=True)  # Tạo thư mục nếu chưa tồn tại

        for file in files:
            input_path = os.path.join(root, file)
            output_path = os.path.join(target_folder, f"face_{file}")

            # Đọc ảnh
            img = cv2.imread(input_path)
            if img is not None:
                face_img = detect_and_crop_face(img)
                if face_img is not None:
                    cv2.imwrite(output_path, face_img)  # Lưu ảnh đã cắt
                    print(f"Đã lưu ảnh: {output_path}")
                else:
                    print(f"Không phát hiện mặt trong ảnh: {input_path}")
            else:
                print(f"Lỗi khi đọc ảnh: {input_path}")

# Chạy hàm xử lý
process_images(input_folder, output_folder)


Đã lưu ảnh: /content/drive/MyDrive/DATN/images_face/ngan/face_image_1_front.jpg
Đã lưu ảnh: /content/drive/MyDrive/DATN/images_face/ngan/face_image_4_top.jpg
Đã lưu ảnh: /content/drive/MyDrive/DATN/images_face/ngan/face_image_3_right.jpg
Đã lưu ảnh: /content/drive/MyDrive/DATN/images_face/ngan/face_image_2_left.jpg
Đã lưu ảnh: /content/drive/MyDrive/DATN/images_face/ngan/face_image_5_bottom.jpg
Đã lưu ảnh: /content/drive/MyDrive/DATN/images_face/khoa/face_image_5_bottom.jpg
Đã lưu ảnh: /content/drive/MyDrive/DATN/images_face/khoa/face_image_1_front.jpg
Đã lưu ảnh: /content/drive/MyDrive/DATN/images_face/khoa/face_image_3_right.jpg
Đã lưu ảnh: /content/drive/MyDrive/DATN/images_face/khoa/face_image_4_top.jpg
Đã lưu ảnh: /content/drive/MyDrive/DATN/images_face/khoa/face_image_2_left.jpg
Đã lưu ảnh: /content/drive/MyDrive/DATN/images_face/phu/face_image_1_front.jpg
Đã lưu ảnh: /content/drive/MyDrive/DATN/images_face/phu/face_image_5_bottom.jpg
Đã lưu ảnh: /content/drive/MyDrive/DATN/image

2. Tăng cường dữ liệu bằng GAN

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import os
from tensorflow.keras.preprocessing.image import load_img, img_to_array, save_img
import matplotlib.pyplot as plt

# Generator Model
def build_generator(latent_dim=256):
    model = tf.keras.Sequential([
        layers.Dense(4 * 4 * 2048, use_bias=False, input_shape=(latent_dim,)),
        layers.BatchNormalization(),
        layers.ReLU(),
        layers.Reshape((4, 4, 2048)),

        layers.Conv2DTranspose(1024, kernel_size=4, strides=2, padding='same', use_bias=False),
        layers.BatchNormalization(),
        layers.ReLU(),

        layers.Conv2DTranspose(512, kernel_size=4, strides=2, padding='same', use_bias=False),
        layers.BatchNormalization(),
        layers.ReLU(),

        layers.Conv2DTranspose(256, kernel_size=4, strides=2, padding='same', use_bias=False),
        layers.BatchNormalization(),
        layers.ReLU(),

        layers.Conv2DTranspose(128, kernel_size=4, strides=2, padding='same', use_bias=False),
        layers.BatchNormalization(),
        layers.ReLU(),

        layers.Conv2DTranspose(3, kernel_size=4, strides=2, padding='same', use_bias=False, activation='tanh')
    ])
    return model

# Discriminator Model
def build_discriminator():
    model = tf.keras.Sequential([
        layers.Conv2D(128, kernel_size=4, strides=2, padding='same', input_shape=(128, 128, 3)),
        layers.LeakyReLU(0.2),

        layers.Conv2D(256, kernel_size=4, strides=2, padding='same'),
        layers.BatchNormalization(),
        layers.LeakyReLU(0.2),

        layers.Conv2D(512, kernel_size=4, strides=2, padding='same'),
        layers.BatchNormalization(),
        layers.LeakyReLU(0.2),

        layers.Conv2D(1024, kernel_size=4, strides=2, padding='same'),
        layers.BatchNormalization(),
        layers.LeakyReLU(0.2),

        layers.Flatten(),
        layers.Dense(1, activation='sigmoid')
    ])
    return model

# Resize input image
def resize_image(image_path, target_size=(128, 128)):
    image = load_img(image_path)
    image = image.resize(target_size)
    image = img_to_array(image)
    image = (image - 127.5) / 127.5  # Normalize to [-1, 1]
    return np.expand_dims(image, axis=0)

# Save generated images
def save_generated_images(generated_images, output_dir):
    # Tạo thư mục đầu ra nếu chưa tồn tại
    os.makedirs(output_dir, exist_ok=True)
    for i, img in enumerate(generated_images):
        # Tạo đường dẫn lưu ảnh
        output_path = os.path.join(output_dir, f"face_aug_{i + 1}.jpg")
        # Chuẩn hóa ảnh từ [-1, 1] về [0, 255]
        img = (img * 127.5 + 127.5).astype(np.uint8)
        # Lưu ảnh ra file JPG
        tf.keras.preprocessing.image.save_img(output_path, img)


# Plot generated images
def plot_generated_images(generator, latent_dim, num_images=16):
    noise = np.random.normal(0, 1, (num_images, latent_dim))
    generated_images = generator.predict(noise)
    generated_images = (generated_images + 1) / 2.0  # Rescale to [0, 1]

    plt.figure(figsize=(8, 8))
    for i in range(num_images):
        # plt.subplot(4, 4, i + 1)
        plt.imshow(generated_images[i])
        plt.axis('off')
    plt.show()

# Training function
def train(generator, discriminator, dataset, latent_dim, epochs=10000, batch_size=64, model_save_path='./models'):
    cross_entropy = tf.keras.losses.BinaryCrossentropy()

    generator_optimizer = tf.keras.optimizers.Adam(1e-4)
    discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

    @tf.function
    def train_step(real_images):
        noise = tf.random.normal([batch_size, latent_dim])

        with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
            generated_images = generator(noise, training=True)

            real_output = discriminator(real_images, training=True)
            fake_output = discriminator(generated_images, training=True)

            gen_loss = cross_entropy(tf.ones_like(fake_output), fake_output)
            real_loss = cross_entropy(tf.ones_like(real_output), real_output)
            fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
            disc_loss = real_loss + fake_loss

        gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
        gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

        generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
        discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

        return gen_loss, disc_loss

    for epoch in range(epochs):
        for image_batch in dataset:
            gen_loss, disc_loss = train_step(image_batch)

        print(f"Epoch {epoch + 1}, Gen Loss: {gen_loss.numpy()}, Disc Loss: {disc_loss.numpy()}")

        # Plot generated images every 10 epochs
        if (epoch + 1) % 10 == 0:
            plot_generated_images(generator, latent_dim)

        # Save model at the end of each epoch
        if (epoch + 1) % epochs == 0:  # Save after every 200 epochs (or after final epoch)
            generator.save(os.path.join(model_save_path, f'generator_epoch_{epoch + 1}.h5'))
            discriminator.save(os.path.join(model_save_path, f'discriminator_epoch_{epoch + 1}.h5'))
            print(f'Models saved after epoch {epoch + 1}')

# Generate 100 images from a single latent vector
def generate_images(generator, latent_dim, num_images=100):
    noise = np.random.normal(0, 1, (num_images, latent_dim))
    generated_images = generator.predict(noise)
    return (generated_images + 1) / 2.0  # Rescale to [0, 1]

# Main execution
if __name__ == "__main__":
    # Base paths
    base_image_path = "/content/drive/MyDrive/DATN/images_face"
    base_output_dir = "/content/drive/MyDrive/DATN/images_face_aug"
    model_save_path = '/content/drive/MyDrive/DATN/models'  # Đường dẫn lưu mô hình

    latent_dim = 100

    for root, dirs, files in os.walk(base_image_path):
        # Lấy đường dẫn thư mục con tương đối
        relative_path = os.path.relpath(root, base_image_path)
        output_folder = os.path.join(base_output_dir, relative_path)
        save_folder = os.path.join(model_save_path, relative_path)

        for file in files:
            # Chỉ xử lý file ảnh
            if file.endswith((".jpg", ".png")):
                image_path = os.path.join(root, file)
                category = os.path.splitext(file)[0].split('_')[-1]
                output_dir = os.path.join(output_folder, category)
                save_dir = os.path.join(save_folder, category)

                # Load và resize ảnh
                input_image = resize_image(image_path)
                input_image = np.squeeze(input_image, axis=0)

                # Build generator và discriminator
                generator = build_generator(latent_dim)
                discriminator = build_discriminator()

                # Chuẩn bị dataset
                dataset = tf.data.Dataset.from_tensor_slices([input_image]).repeat(100).batch(16)

                # Train GAN
                train(generator, discriminator, dataset, latent_dim, epochs=200, batch_size=16, model_save_path=save_dir)

                # Generate images
                generated_images = generate_images(generator, latent_dim, num_images=100)

                # Save generated images
                save_generated_images(generated_images, output_dir)

                print(f"Generated images saved for {image_path}.")

3. Huấn luyện mô hình nhận dạng khuôn mặt dùng kiến trúc MobileNetV2 từ dữ liệu đã tăng cường

3.1. Tạo dataset và chia tập train/val

In [ ]:
import os
import shutil

# Đường dẫn thư mục gốc
base_input_dir = "/content/drive/MyDrive/DATN/images_face_aug"
base_output_dir = "/content/drive/MyDrive/DATN/dataset"

def copy_and_rename_images(root_folder, output_folder, max_images=500):
    # Tạo thư mục đích nếu chưa tồn tại
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Duyệt qua từng thư mục con (tương ứng với từng người)
    for person_folder in os.listdir(root_folder):
        person_path = os.path.join(root_folder, person_folder)
        if os.path.isdir(person_path):
            # Tạo thư mục mới cho người này trong output_folder
            person_output_folder = os.path.join(output_folder, person_folder)
            if not os.path.exists(person_output_folder):
                os.makedirs(person_output_folder)

            # Thu thập tất cả các ảnh từ các thư mục con
            image_paths = []
            for subfolder in os.listdir(person_path):
                subfolder_path = os.path.join(person_path, subfolder)
                if os.path.isdir(subfolder_path):
                    for file in os.listdir(subfolder_path):
                        if file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
                            image_paths.append(os.path.join(subfolder_path, file))

            # Giới hạn số lượng ảnh tối đa và sao chép ảnh
            image_paths = image_paths[:max_images]
            for i, image_path in enumerate(image_paths, start=1):
                output_path = os.path.join(person_output_folder, f"{i}.jpg")
                shutil.copy(image_path, output_path)
                print(f"Đã sao chép {image_path} -> {output_path}")

# Cấu hình
root_folder = "/content/drive/MyDrive/DATN/images_face_aug"
output_folder = "/content/drive/MyDrive/DATN/dataset"

copy_and_rename_images(root_folder, output_folder)



Đã sao chép /content/drive/MyDrive/DATN/images_face_aug/ngan/front/face_aug_1.jpg -> /content/drive/MyDrive/DATN/dataset/ngan/1.jpg
Đã sao chép /content/drive/MyDrive/DATN/images_face_aug/ngan/front/face_aug_2.jpg -> /content/drive/MyDrive/DATN/dataset/ngan/2.jpg
Đã sao chép /content/drive/MyDrive/DATN/images_face_aug/ngan/front/face_aug_3.jpg -> /content/drive/MyDrive/DATN/dataset/ngan/3.jpg
Đã sao chép /content/drive/MyDrive/DATN/images_face_aug/ngan/front/face_aug_4.jpg -> /content/drive/MyDrive/DATN/dataset/ngan/4.jpg
Đã sao chép /content/drive/MyDrive/DATN/images_face_aug/ngan/front/face_aug_5.jpg -> /content/drive/MyDrive/DATN/dataset/ngan/5.jpg
Đã sao chép /content/drive/MyDrive/DATN/images_face_aug/ngan/front/face_aug_6.jpg -> /content/drive/MyDrive/DATN/dataset/ngan/6.jpg
Đã sao chép /content/drive/MyDrive/DATN/images_face_aug/ngan/front/face_aug_7.jpg -> /content/drive/MyDrive/DATN/dataset/ngan/7.jpg
Đã sao chép /content/drive/MyDrive/DATN/images_face_aug/ngan/front/face_aug_

In [ ]:
import os
import shutil
from PIL import Image
from sklearn.model_selection import train_test_split

def resize_image(image_path, output_path, size=(160, 160)):
    try:
        with Image.open(image_path) as img:
            img = img.resize(size, Image.Resampling.LANCZOS)
            img.save(output_path)
    except Exception as e:
        print(f"Không thể resize ảnh {image_path}: {e}")

def split_and_resize_images(input_folder, output_folder, size=(160, 160), train_ratio=0.8):
    # Tạo các thư mục train và val
    train_folder = os.path.join(output_folder, "train")
    val_folder = os.path.join(output_folder, "val")
    os.makedirs(train_folder, exist_ok=True)
    os.makedirs(val_folder, exist_ok=True)

    for person in os.listdir(input_folder):
        person_path = os.path.join(input_folder, person)
        if os.path.isdir(person_path):
            # Tạo thư mục cho từng người trong train và val
            person_train_folder = os.path.join(train_folder, person)
            person_val_folder = os.path.join(val_folder, person)
            os.makedirs(person_train_folder, exist_ok=True)
            os.makedirs(person_val_folder, exist_ok=True)

            # Lấy danh sách tất cả ảnh
            image_paths = [
                os.path.join(person_path, img)
                for img in os.listdir(person_path)
                if img.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif'))
            ]

            # Chia tỉ lệ train/val
            train_paths, val_paths = train_test_split(image_paths, train_size=train_ratio, random_state=42)

            # Resize và sao chép ảnh vào train
            for img_path in train_paths:
                output_path = os.path.join(person_train_folder, os.path.basename(img_path))
                resize_image(img_path, output_path, size)

            # Resize và sao chép ảnh vào val
            for img_path in val_paths:
                output_path = os.path.join(person_val_folder, os.path.basename(img_path))
                resize_image(img_path, output_path, size)

            print(f"Hoàn tất cho {person}: {len(train_paths)} train, {len(val_paths)} val.")

# Cấu hình
input_folder = "/content/drive/MyDrive/DATN/dataset"  # Thư mục chứa ảnh đã sắp xếp (theo tên người)
output_folder = "/content/drive/MyDrive/DATN/dataset_splitted"  # Thư mục đích để lưu dữ liệu train/val
split_and_resize_images(input_folder, output_folder)


Hoàn tất cho ngan: 400 train, 100 val.
Hoàn tất cho khoa: 400 train, 100 val.
Hoàn tất cho phu: 400 train, 100 val.


3.2. Xây dựng mô hình + train

In [10]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
import cv2
import os
import numpy as np
from sklearn.model_selection import train_test_split

# Path to your dataset
data_dir = '/content/drive/MyDrive/DATN/dataset'

def preprocess_images(data_dir):
    images = []
    labels = []
    for label, person in enumerate(os.listdir(data_dir)):
        person_path = os.path.join(data_dir, person)
        for image_file in os.listdir(person_path):
            image_path = os.path.join(person_path, image_file)
            image = cv2.imread(image_path)
            image = cv2.resize(image, (224, 224))  # Resize to 224x224
            image = image / 255.0  # Normalize pixel values to [0, 1]
            images.append(image)
            labels.append(label)
    return np.array(images), np.array(labels)

# Preprocess images
images, labels = preprocess_images(data_dir)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

Define the model
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(3, activation='softmax')(x)  # 3 classes for face recognition
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze base model layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Save the model
model.save('face_recognition_model.h5')

# Convert the model to TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]  # Apply quantization
tflite_model = converter.convert()

# Save the TensorFlow Lite model
with open('face_recognition_model.tflite', 'wb') as f:
    f.write(tflite_model)

print("Model converted to TensorFlow Lite format!")

# Test the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)


In [11]:
from tensorflow.keras.models import load_model
import tensorflow as tf


# Tải mô hình Keras từ tệp .h5
model = load_model('/content/drive/MyDrive/DATN/face_recognition_model.h5')

# Convert the model to TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_keras_model(model)

# Apply default optimization for quantization
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# Provide a representative dataset for calibration
def representative_data_gen():
    for image in X_train[:100]:  # Use 100 samples for calibration
        image = np.expand_dims(image, axis=0).astype(np.float32)
        yield [image]

converter.representative_dataset = representative_data_gen

# Ensure compatibility with integer-only hardware (e.g., EdgeTPU)
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]

# Convert the model
tflite_model = converter.convert()

# Save the TensorFlow Lite model
with open('face_recognition_model_quantized.tflite', 'wb') as f:
    f.write(tflite_model)

print("Quantized model successfully converted and saved!")


Saved artifact at '/tmp/tmp_yy2xeq7'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 3), dtype=tf.float32, name=None)
Captures:
  139182428987712: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139182428819120: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139182428821232: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139182428994576: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139182428816832: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139182428824752: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139182428825104: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139182428826512: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139182428822992: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139182428824400: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139182428828800

/usr/local/lib/python3.10/dist-packages/tensorflow/lite/python/convert.py:983: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


Quantized model successfully converted and saved!


In [12]:
import tensorflow as tf

# Load the TFLite model
tflite_model_path = '/content/drive/MyDrive/DATN/face_recognition_model_quantized.tflite'

# Initialize the TFLite interpreter
interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
interpreter.allocate_tensors()

# Get model details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Print input details
print("Input Details:")
for detail in input_details:
    print(f"Name: {detail['name']}")
    print(f"Shape: {detail['shape']}")
    print(f"Type: {detail['dtype']}\n")

# Print output details
print("Output Details:")
for detail in output_details:
    print(f"Name: {detail['name']}")
    print(f"Shape: {detail['shape']}")
    print(f"Type: {detail['dtype']}\n")


Input Details:
Name: serving_default_input_layer:0
Shape: [  1 224 224   3]
Type: <class 'numpy.float32'>

Output Details:
Name: StatefulPartitionedCall_1:0
Shape: [1 3]
Type: <class 'numpy.float32'>

